In [1]:
import torch
torch.manual_seed(0)
from PIL import Image
import json
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
import re

def clean_text(text):
    text = re.sub(r'(\\n)+', '. ', text)
    text = text.replace("\\", " ")

    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [3]:
# Mapping of labels to indices
label_dict = {
    'Appeal to (Strong) Emotions': 0,
    'Appeal to authority': 1,
    'Appeal to fear/prejudice': 2,
    'Bandwagon': 3,
    'Black-and-white Fallacy/Dictatorship': 4,
    'Causal Oversimplification': 5,
    'Doubt': 6,
    'Exaggeration/Minimisation': 7,
    'Flag-waving': 8,
    'Glittering generalities (Virtue)': 9,
    'Loaded Language': 10,
    "Misrepresentation of Someone's Position (Straw Man)": 11,
    'Name calling/Labeling': 12,
    'Obfuscation, Intentional vagueness, Confusion': 13,
    'Presenting Irrelevant Data (Red Herring)': 14,
    'Reductio ad hitlerum': 15,
    'Repetition': 16,
    'Slogans': 17,
    'Smears': 18,
    'Thought-terminating cliche': 19,
    'Transfer': 20,
    'Whataboutism': 21
}

In [4]:
persuasion_techniques = [key for key, value in label_dict.items()]

In [5]:
class MemesDataset(Dataset):
    def __init__(self, json_file, img_dir, label_dict, transform=None):
        with open(json_file, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.img_dir = img_dir
        self.label_dict = label_dict
        self.num_labels = len(label_dict)
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img_path = f"{self.img_dir}/{item['image']}"
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        text = clean_text(item['text'])
        # Create a zero vector of length num_labels
        label_vector = torch.zeros(self.num_labels, dtype=torch.float32)
        # Set positions of labels to 1
        # labels = [label.replace('é', 'e') for label in item['labels']]
        for label in item['labels']:
            if label in self.label_dict:
                label_idx = self.label_dict[label]
                label_vector[label_idx] = 1
        
        # print(label_vector)
        return image, text, label_vector


In [6]:
!pip install transformers


Defaulting to user installation because normal site-packages is not writeable


In [7]:
from transformers import CLIPModel
import torch.nn as nn

class CLIPClassifier(nn.Module):
    def __init__(self, num_labels):
        super(CLIPClassifier, self).__init__()
        self.clip = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
        # Get the output dimension of the text and image embeddings from the model's configuration
        embedding_dim = self.clip.config.projection_dim
        self.classifier = nn.Linear(embedding_dim * 2, num_labels)  # Assuming concatenation of embeddings

    def forward(self, input_ids, pixel_values, attention_mask):
        outputs = self.clip(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask)
        # Concatenate text and image embeddings along the last dimension
        combined_features = torch.cat((outputs.text_embeds, outputs.image_embeds), dim=-1)
        logits = self.classifier(combined_features)
        return logits

C:\Users\harih\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from transformers import CLIPProcessor

# Initialize the dataset and dataloader
img_dir = 'train_images/train_images'
json_file = 'annotations_v2/semeval2024_dev_release/subtask2a/train.json'
dataset = MemesDataset(json_file, img_dir, label_dict)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize the processor and model
processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')
# model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
model = CLIPClassifier(num_labels=22)
model.train()

CLIPClassifier(
  (clip): CLIPModel(
    (text_model): CLIPTextTransformer(
      (embeddings): CLIPTextEmbeddings(
        (token_embedding): Embedding(49408, 512)
        (position_embedding): Embedding(77, 512)
      )
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-11): 12 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELUActivation()
              (fc1): Linear(in_features=512, out_features=2048, bias=True)
              (fc2): Linear(in_features=2048, out_features=512, bias=True)
 

In [9]:
import torch.optim as optim
from tqdm import tqdm


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4)
epochs = 100
loss_fn = torch.nn.BCEWithLogitsLoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, texts, labels in tqdm(dataloader):
        # print(labels)
        inputs = processor(text=texts, images=images, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Ensure the model forward method aligns with inputs correctly
        logits = model(input_ids=inputs['input_ids'], pixel_values=inputs['pixel_values'], attention_mask=inputs['attention_mask'])

        loss = loss_fn(logits, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Average Loss: {total_loss / len(dataloader)}")


100%|██████████| 110/110 [19:00<00:00, 10.37s/it]


Epoch 1, Average Loss: 0.5552657029845498


100%|██████████| 110/110 [18:14<00:00,  9.95s/it]


Epoch 2, Average Loss: 0.43526397130706096


100%|██████████| 110/110 [18:18<00:00,  9.98s/it]


...

Epoch 98, Average Loss: 0.014454013752666387


100%|██████████| 110/110 [18:20<00:00, 10.00s/it]


Epoch 99, Average Loss: 0.014049320244653659


100%|██████████| 110/110 [18:40<00:00, 10.18s/it]

Epoch 100, Average Loss: 0.013693082417276773


In [10]:
model.eval()

# processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')
import os

def load_and_preprocess_data(entry, img_dir, processor, max_length=77):
    # Load the image
    image_path = os.path.join(img_dir, entry['image'])
    image = Image.open(image_path).convert("RGB")
    
    # Truncate text to the maximum length allowed by CLIP
    text = entry['text']
    inputs = processor(text=[text[:max_length]], images=[image], return_tensors="pt", padding=True, truncation=True)
    return inputs


def perform_inference(model, processor, data, img_dir, threshold=0.4):
    results = []
    for entry in tqdm(data):
        try:
            inputs = load_and_preprocess_data(entry, img_dir, processor)
            inputs = {key: val.to(device) for key, val in inputs.items()}  # Move to GPU if available

            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs  # Assuming logits are being correctly handled as per your model
                probs = torch.sigmoid(logits).squeeze()
                # print(probs)
                # print(probs)
                predicted_labels = probs > threshold
                # print(predicted_labels)
                # print(predicted_labels_indices)
                predicted_labels_indices = predicted_labels.nonzero()
                # print(predicted_labels_indices)
                
                predicted_labels_indices = predicted_labels_indices.cpu().squeeze().numpy()
                
                
                # print(predicted_labels_indices, type(predicted_labels_indices))
                
                if predicted_labels_indices.ndim == 0:
                    predicted_labels_indices = [predicted_labels_indices.item()]
                
                # print(predicted_labels_indices)
                predicted_labels = [persuasion_techniques[idx] for idx in predicted_labels_indices]
                

            results.append({
                "id": entry['id'],
                "labels": predicted_labels
            })
        except Exception as e:
            print(f"Error processing entry {entry['id']}: {str(e)}")
            results.append({
                "id": entry['id'],
                "labels": []
            })
        
    return results



In [11]:
def load_data(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

data = load_data('annotations_v2/semeval2024_dev_release/subtask2a/dev_unlabeled.json')
img_dir = 'dev_images/dev_images'


results = perform_inference(model, processor, data, img_dir)

# Saving the results to a JSON file
with open('predicted_labels_clip_devUn.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4)

100%|██████████| 1000/1000 [00:59<00:00, 16.67it/s]


In [12]:
with open('predicted_labels_clip_devUn.json', 'r', encoding='utf-8') as f:
    pdata = f.read()
    print(pdata)


[
    {
        "id": "63866",
        "labels": [
            "Glittering generalities (Virtue)"
        ]
    },
    {
        "id": "78505",
        "labels": [
            "Flag-waving",
            "Name calling/Labeling",
            "Slogans",
            "Smears"
        ]
    },
    {
        "id": "65750",
        "labels": [
            "Smears"
        ]
    },
    {
        "id": "78716",
        "labels": [
            "Smears"
        ]
    },
    {
        "id": "67988",
        "labels": [
            "Black-and-white Fallacy/Dictatorship",
            "Loaded Language"
        ]
    },
    {
        "id": "65792",
        "labels": [
            "Smears"
        ]
    },
    {
        "id": "66205",
        "labels": [
            "Smears"
        ]
    },
    {
        "id": "65097",
        "labels": [
            "Smears"
        ]
    },
    {
        "id": "77523",
        "labels": [
            "Repetition",
            "Smears"
        ]
    },
    {
        "

In [13]:
!pip install numpy scikit-learn networkx sklearn-hierarchical-classification


Defaulting to user installation because normal site-packages is not writeable


In [14]:
! python subtask_1_2a.py -g dev_gold_labels\dev_gold_labels\dev_subtask2a_en.json -p predicted_labels_clip_devUn.json

f1_h=0.52785	prec_h=0.60858	rec_h=0.46603


In [15]:
data = load_data('test_data/test_data/north_macedonian/mk_subtask2a_test_unlabeled.json')
img_dir = 'test_images/test_images/subtask1_2a/north_macedonian'

results = perform_inference(model, processor, data, img_dir)

# Saving the results to a JSON file
with open('clip100_mk_predictions.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4)

100%|██████████| 259/259 [00:14<00:00, 17.60it/s]


In [16]:
!python subtask_1_2a.py -g gold_labels_ar_bg_md_version2/test_subtask2a_md.json -p clip100_mk_predictions.json

f1_h=0.62421	prec_h=0.69522	rec_h=0.56636


In [17]:
data = load_data('test_data/test_data/bulgarian/bg_subtask2a_test_unlabeled.json')
img_dir = 'test_images/test_images/subtask1_2a/bulgarian'

results = perform_inference(model, processor, data, img_dir)

# Saving the results to a JSON file
with open('clip100_bg_predictions.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4)

100%|██████████| 436/436 [00:24<00:00, 17.46it/s]


In [18]:
!python subtask_1_2a.py -g gold_labels_ar_bg_md_version2/test_subtask2a_bg.json -p clip100_bg_predictions.json

f1_h=0.55658	prec_h=0.59778	rec_h=0.52069


In [19]:
data = load_data('test_data_arabic/test_data_arabic/ar_subtask2a_test_unlabeled.json')
img_dir = 'test_images_arabic/test_images_arabic/subtask2a'

results = perform_inference(model, processor, data, img_dir)

# Saving the results to a JSON file
with open('clip100_ar_predictions.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4)

100%|██████████| 120/120 [00:06<00:00, 17.72it/s]


In [20]:
!python subtask_1_2a.py -g gold_labels_ar_bg_md_version2/test_subtask2a_ar.json -p clip100_ar_predictions.json

f1_h=0.49914	prec_h=0.51408	rec_h=0.48505


In [21]:
data = load_data('test_data/test_data/english/en_subtask2a_test_unlabeled.json')
img_dir = 'test_images/test_images/subtask1_2a/english'

results = perform_inference(model, processor, data, img_dir, 0.3)

# Saving the results to a JSON file
with open('clip100_en_predictions.json.txt', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4)

100%|██████████| 1500/1500 [01:26<00:00, 17.40it/s]


In [22]:
# English test scores
#Hierarchical F1	Hierarchical Precision	Hierarchical Recall
# 0.52467	               0.67117          	0.43067

In [23]:
torch.save(model.state_dict(), 'CLIP100_subtask2a.pth')